## TASK 3: Bell States and Entanglement Entropy  

In [ ]:
#!pip install qiskit
import numpy as np
from math import log2, sqrt

In [ ]:
# Define quantum gates
H = 1/sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate
I = np.eye(2)                                # Identity gate
CNOT = np.array([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]])  # CNOT gate

In [ ]:
class BellStates:
    @staticmethod
    def phi_plus():
        """Construct |Φ⁺⟩ = (|00⟩ + |11⟩)/√2"""
        state = np.kron([1, 0], [1, 0])  # |00⟩
        state = np.kron(H, I) @ state    # Apply H to first qubit
        return CNOT @ state              # Apply CNOT
    
    @staticmethod
    def phi_minus():
        """Construct |Φ⁻⟩ = (|00⟩ - |11⟩)/√2"""
        state = np.kron([0, 1], [1, 0])  # |10⟩
        state = np.kron(H, I) @ state
        return CNOT @ state
    
    @staticmethod
    def psi_plus():
        """Construct |Ψ⁺⟩ = (|01⟩ + |10⟩)/√2"""
        state = np.kron([1, 0], [0, 1])  # |01⟩
        state = np.kron(H, I) @ state
        return CNOT @ state
    
    @staticmethod
    def psi_minus():
        """Construct |Ψ⁻⟩ = (|01⟩ - |10⟩)/√2"""
        state = np.kron([0, 1], [0, 1])  # |11⟩
        state = np.kron(H, I) @ state
        return CNOT @ state

In [ ]:
def partial_trace(rho, dims, axis=0):
    """
    Compute partial trace of density matrix rho
    dims: list of dimensions of each subsystem [dA, dB]
    axis: 0 for tracing out B, 1 for tracing out A
    """
    dA, dB = dims
    if axis == 0:  # Trace out B
        rho_reduced = np.zeros((dA, dA), dtype=complex)
        for i in range(dA):
            for j in range(dA):
                for k in range(dB):
                    rho_reduced[i,j] += rho[i*dB + k, j*dB + k]
    else:  # Trace out A
        rho_reduced = np.zeros((dB, dB), dtype=complex)
        for i in range(dB):
            for j in range(dB):
                for k in range(dA):
                    rho_reduced[i,j] += rho[k*dB + i, k*dB + j]
    return rho_reduced


In [ ]:
def entanglement_entropy(state):
    """
    Calculate entanglement entropy of bipartite state
    Input: state vector or density matrix
    Output: entanglement entropy
    """
    # Convert state to density matrix if it's a state vector
    if state.ndim == 1:
        rho = np.outer(state, state.conj())
    else:
        rho = state
    
     # Partial trace over subsystem B (assuming 2-qubit system)
    rho_A = partial_trace(rho, [2, 2], axis=1)
    
    # Compute eigenvalues (using eigh for Hermitian matrices)
    eigvals = np.linalg.eigvalsh(rho_A)
    
    # Calculate von Neumann entropy
    entropy = 0.0
    for lamda in eigvals:
        if lamda > 1e-10:  # avoid log(0)
            entropy -= lamda * log2(lamda)
    
    return entropy

In [ ]:
   # Example usage
if __name__ == "__main__":
    # Construct Bell states
    phi_p = BellStates.phi_plus()
    phi_m = BellStates.phi_minus()
    psi_p = BellStates.psi_plus()
    psi_m = BellStates.psi_minus()

    print("\n" + "="*50)
    print("TASK 3: BELL STATES AND ENTANGLEMENT ENTROPY")
    print("="*50)

    print(f"Bell state |Φ⁺⟩ =", phi_p)
    print(f"Bell state |Φ⁻⟩ =", phi_m)
    print(f"Bell state |Ψ⁺⟩ =", psi_p)
    print(f"Bell state |Ψ⁻⟩ =", psi_m)

   # Verify entanglement entropy (should be 1 for maximally entangled states)
    print(f"Entanglement entropy of |Φ⁺⟩: {entanglement_entropy(phi_p):.4f}")
    print(f"Entanglement entropy of |Φ⁻⟩: {entanglement_entropy(phi_m):.4f}")
    print(f"Entanglement entropy of |Ψ⁺⟩: {entanglement_entropy(psi_p):.4f}")
    print(f"Entanglement entropy of |Ψ⁻⟩: {entanglement_entropy(psi_m):.4f}")
    
    # Verify product state has zero entanglement entropy
    product_state = np.kron([1, 0], [1, 0])  # |00⟩
    print(f"Entanglement entropy of |00⟩: {entanglement_entropy(product_state):.4f}")


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, entropy
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd

# Initialize simulator
simulator = AerSimulator()

def create_bell_state(bell_type):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    if bell_type == 'phi_minus':
        qc.z(0)
    elif bell_type == 'psi_plus':
        qc.x(1)
    elif bell_type == 'psi_minus':
        qc.z(0)
        qc.x(1)
    return qc

def calculate_entropy(qc):
    statevector = Statevector.from_instruction(qc)
    rho = partial_trace(statevector, [1])
    return entropy(rho, base=2)

state_data = []
states = {
    'phi_plus': '|Φ⁺⟩',
    'phi_minus': '|Φ⁻⟩',
    'psi_plus': '|Ψ⁺⟩',
    'psi_minus': '|Ψ⁻⟩',
    'product': '|00⟩'
}

print("Bell States Construction and Entanglement Analysis")
print("="*50)

for key, label in states.items():
    if key == 'product':
        qc = QuantumCircuit(2)
        print("\nProduct State:")
    else:
        qc = create_bell_state(key)
        print(f"\nBell State {label}:")
    
    # Display circuit
    print("Quantum Circuit:")
    print(qc)
    
    # Calculate and display results
    entropy_val = calculate_entropy(qc)
    statevector = Statevector.from_instruction(qc)
    
    print("\nStatevector:")
    print(np.round(statevector.data, 3))
    
    print(f"Entanglement Entropy: {round(entropy_val, 3)}")
    print("-"*40)
    
    state_data.append({
        'State': label,
        'Entanglement Entropy': round(entropy_val, 3),
        'Score (out of 1)': f"{round(entropy_val, 3)}/1"
    })

# Create and display summary table
print("\nSummary Table:")
print("="*50)
df = pd.DataFrame(state_data)
print(df.to_string(index=False))